In [0]:
# needs to be installed once
# !pip install pysus

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.3/792.3 kB 73.2 MB/s eta 0:00:00
  Created wheel for pyreaddbc: filename=pyreaddbc-1.2.0-cp311-cp311-manylinux_2_35_aarch64.whl size=89609 sha256=c18068aee29541a69cfb9b399132e0db9e183132ffcc04e58bb686c30301b3d

In [0]:
# restart after installation
# dbutils.library.restartPython()

In [0]:
# libraries
from pysus.online_data.SINASC import download
from pyspark.sql import functions as F

from functools import reduce

In [0]:
# states by region
regions = {
    "norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    "nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "centro_oeste": ["DF", "GO", "MT", "MS"],
    "sudeste": ["ES", "MG", "RJ", "SP"],
    "sul": ["PR", "RS", "SC"]
}

years = [2020]

# loop for state and years
dfs_region_year = {}

for region, states in regions.items():
    for year in years:
        dfs_spark = []
        
        for state in states:
            # pandas -> spark
            df_pd = download(groups='DN', states=state, years=year).to_dataframe()
            df_pd['STATE'] = state
            df_pd['YEAR'] = year
            dfs_spark.append(spark.createDataFrame(df_pd))
        
        # concatenate all states in the region in the year
        df_spark_region_year = reduce(
            lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), 
            dfs_spark
        )
        
        # save in dictionary with clear name
        key = f"{region}_{year}"
        dfs_region_year[key] = df_spark_region_year
        
        table_name = f"workspace.triggo.{key}_raw"
        df_spark_region_year.write.format("delta").mode("overwrite").saveAsTable(table_name)
        
        print(f"✅ Spark dataframe created and saved: {key} ({df_spark_region_year.count()} records)")



DNAC2020.parquet: 100%|██████████| 17.0k/17.0k [00:00<00:00, 20.4kB/s]
DNAP2020.parquet: 100%|██████████| 16.4k/16.4k [00:00<00:00, 22.6kB/s]
DNAM2020.parquet: 100%|██████████| 84.7k/84.7k [00:04<00:00, 19.1kB/s]
DNPA2020.parquet: 100%|██████████| 149k/149k [00:07<00:00, 20.9kB/s]
DNRO2020.parquet: 100%|██████████| 28.9k/28.9k [00:01<00:00, 21.5kB/s]
DNRR2020.parquet: 100%|██████████| 15.4k/15.4k [00:00<00:00, 20.0kB/s]
DNTO2020.parquet: 100%|██████████| 26.6k/26.6k [00:01<00:00, 20.6kB/s]


✅ Spark dataframe created and saved: norte_2020 (301635 records)


DNAL2020.parquet: 100%|██████████| 54.2k/54.2k [00:02<00:00, 21.0kB/s]
DNBA2020.parquet: 100%|██████████| 212k/212k [00:10<00:00, 19.9kB/s]
DNCE2020.parquet: 100%|██████████| 137k/137k [00:06<00:00, 20.8kB/s]
DNMA2020.parquet: 100%|██████████| 119k/119k [00:09<00:00, 13.1kB/s]
DNPB2020.parquet: 100%|██████████| 63.2k/63.2k [00:02<00:00, 21.7kB/s]
DNPE2020.parquet: 100%|██████████| 144k/144k [00:07<00:00, 20.2kB/s]
DNPI2020.parquet: 100%|██████████| 50.7k/50.7k [00:02<00:00, 20.1kB/s]
DNRN2020.parquet: 100%|██████████| 48.8k/48.8k [00:02<00:00, 21.0kB/s]
DNSE2020.parquet: 100%|██████████| 35.6k/35.6k [00:01<00:00, 20.4kB/s]


✅ Spark dataframe created and saved: nordeste_2020 (770688 records)


DNDF2020.parquet: 100%|██████████| 44.1k/44.1k [00:02<00:00, 20.0kB/s]
DNGO2020.parquet: 100%|██████████| 104k/104k [00:05<00:00, 20.1kB/s]
DNMT2020.parquet: 100%|██████████| 63.9k/63.9k [00:02<00:00, 21.7kB/s]
DNMS2020.parquet: 100%|██████████| 46.3k/46.3k [00:02<00:00, 21.6kB/s]


✅ Spark dataframe created and saved: centro_oeste_2020 (230474 records)


DNES2020.parquet: 100%|██████████| 60.2k/60.2k [00:03<00:00, 18.6kB/s]
DNMG2020.parquet: 100%|██████████| 277k/277k [00:12<00:00, 21.7kB/s]
DNRJ2020.parquet: 100%|██████████| 223k/223k [00:10<00:00, 21.3kB/s]
DNSP2020.parquet: 100%|██████████| 619k/619k [00:32<00:00, 19.2kB/s]


✅ Spark dataframe created and saved: sudeste_2020 (1052399 records)


DNPR2020.parquet: 100%|██████████| 164k/164k [00:07<00:00, 21.5kB/s]
DNRS2020.parquet: 100%|██████████| 146k/146k [00:06<00:00, 21.4kB/s]
DNSC2020.parquet: 100%|██████████| 110k/110k [00:05<00:00, 21.4kB/s]


✅ Spark dataframe created and saved: sul_2020 (374949 records)


In [0]:
df_sudeste_2022 = spark.read.table("workspace.triggo.sudeste_2022")
df_sudeste_2022.limit(5).display()


ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,CODMUNRES,GESTACAO,GRAVIDEZ,PARTO,CONSULTAS,DTNASC,HORANASC,SEXO,APGAR1,APGAR5,RACACOR,PESO,IDANOMAL,DTCADASTRO,CODANOMAL,NUMEROLOTE,VERSAOSIST,DTRECEBIM,DIFDATA,DTRECORIGA,NATURALMAE,CODMUNNATU,CODUFNATU,ESCMAE2010,SERIESCMAE,DTNASCMAE,RACACORMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,IDADEPAI,DTULTMENST,SEMAGESTAC,TPMETESTIM,CONSPRENAT,MESPRENAT,TPAPRESENT,STTRABPART,STCESPARTO,TPNASCASSI,TPFUNCRESP,TPDOCRESP,DTDECLARAC,ESCMAEAGR1,STDNEPIDEM,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK,CONTADOR,STATE,YEAR
1,6943284,350320,1,35,2,4,421125,03,00,350320,4,2,2,4,09092020,0144,1,09,10,1,2240,2,21092020,,20200018,3.2.01,30092020,021,,835,350320,35,3,3,26121984,1,03,03,00,31,29122019,36,8,09,03,2,2,1,1,2,4,09092020,06,0,1,1,08,1,5,2180909,SP,2020
1,3625087,350320,1,36,5,5,422105,00,02,350320,5,1,2,4,17092020,0732,1,10,10,1,4245,2,21092020,,20200018,3.2.01,30092020,013,,835,350320,35,5,,04061984,1,02,00,00,37,26122019,37,8,10,01,1,2,1,1,2,4,17092020,08,0,1,1,04,1,5,2180910,SP,2020
1,3625087,350320,1,31,5,5,411010,00,00,350320,5,1,2,4,11092020,1518,1,08,09,1,4030,2,21092020,,20200018,3.2.01,30092020,019,,835,350320,35,5,,13071989,1,00,00,00,28,06122019,39,8,12,01,1,2,2,1,2,3,11092020,08,0,1,1,01,0,5,2180911,SP,2020
1,3625087,350320,1,33,5,4,141410,03,00,350320,5,1,2,4,11092020,0727,1,09,10,3,3030,2,21092020,,20200018,3.2.01,30092020,019,,,,,3,3,12051987,3,03,03,00,40,08122019,39,8,07,02,1,2,2,1,2,3,11092020,06,0,1,1,03,1,5,2180912,SP,2020
1,3625087,350320,1,31,4,4,421125,00,00,350320,5,1,2,3,15092020,1635,1,09,10,1,3650,2,21092020,,20200018,3.2.01,30092020,015,,842,421720,42,3,3,20021989,1,00,00,00,46,23122019,37,8,06,02,1,2,2,1,2,3,15092020,06,0,1,1,01,0,4,2180913,SP,2020
